In [224]:
import WAN
import os
import mat73
dataset = 'authorshipData.mat' # store the human paper
# here the aim is to convert the text into WAN

In [225]:
import json
gpt_response = "responses.json"
gpt_papers = []
with open(gpt_response,'r') as f:
    response = f.readline()
    gpt_papers.append(json.loads(response))


# show the basic structure of original human expercet

In [226]:
human_data = mat73.loadmat(dataset)


In [227]:
print(human_data.keys())
for i in human_data["all_wans"]:
    print(i.shape)
print(human_data['function_words'])

for i in human_data["all_freqs"]:
    print(i.shape)
print(human_data["all_wans"][0][:,:,3][0],human_data["all_freqs"][0][:,3])

dict_keys(['all_authors', 'all_freqs', 'all_wans', 'function_words'])
(211, 211, 337)
(211, 211, 448)
(211, 211, 284)
(211, 211, 328)
(211, 211, 467)
(211, 211, 771)
(211, 211, 682)
(211, 211, 907)
(211, 211, 936)
(211, 211, 626)
(211, 211, 405)
(211, 211, 439)
(211, 211, 1022)
(211, 211, 299)
(211, 211, 212)
(211, 211, 595)
(211, 211, 415)
(211, 211, 613)
(211, 211, 423)
(211, 211, 366)
(211, 211, 555)
[['a'], ['bit'], ['couple'], ['aboard'], ['about'], ['above'], ['absent'], ['according'], ['accordingly'], ['across'], ['after'], ['against'], ['ahead'], ['albeit'], ['all'], ['along'], ['alongside'], ['although'], ['amid'], ['amidst'], ['among'], ['amongst'], ['an'], ['and'], ['another'], ['any'], ['anybody'], ['anyone'], ['anything'], ['around'], ['as'], ['aside'], ['astraddle'], ['astride'], ['at'], ['away'], ['bar'], ['barring'], ['because'], ['before'], ['behind'], ['below'], ['beneath'], ['beside'], ['besides'], ['between'], ['beyond'], ['both'], ['but'], ['by'], ['can'], ['certai

In [228]:
print(human_data["all_authors"])

[['abbott'], ['stevenson'], ['alcott'], ['alger'], ['allen'], ['austen'], ['bronte'], ['cooper'], ['dickens'], ['garland'], ['hawthorne'], ['james'], ['melville'], ['page'], ['thoreau'], ['twain'], ['doyle'], ['irving'], ['poe'], ['jewett'], ['wharton']]


In [229]:
human_data.keys()
print(human_data['all_wans'][0].shape)
function_words = [word[0] for word in human_data['function_words']]
print(len(function_words))


(211, 211, 337)
224


In [230]:
human_data['all_freqs'][0].shape

(211, 337)

# load chatgpt expercerts

In [238]:
def load_gpt(function_words,windowWidth,file = "responses.json"):
    gpt_response = file
    gpt_papers = []
    WANS = []
    
    wordcounts = []
    with open(gpt_response,'r') as f:
        while(True):
            response = f.readline()
            if response:
                try:
                    gpt_papers.append(json.loads(response))
                except:
                    print(response)
                    
            else:
                break
    for paper in gpt_papers:
        #paper = json.loads()
        papers = paper.items()
        for writer,novel in papers:
            
            # generate WAN for novel and store it in to gptWANs
            (text1Scores, text1Counts) = WAN.countFWs(WAN.cleanUp(novel),function_words,windowWidth)
            WAN1 = WAN.normalize(text1Scores) 
            
            WANS.append(WAN1)
            wordcounts.append(text1Counts)
    return wordcounts, WANS
            
            

In [239]:
wordcounts, WANS = load_gpt(function_words[:211],5) # obtain wans and word counts last 13 word is unnecessary

{"emily bronte": "As the wind howled through the moors, I found myself lost in a sea of emotions. The vast expanse of the heath, with its rolling hills and endless skies, seemed to mirror the tumultuous turmoil that raged within my heart.\n\nI had always been a solitary soul, content to wander the wilds of nature and lose myself in the beauty of the world around me. But now, as I stood on the edge of the cliff, staring out into the abyss, I couldn't help but feel a sense of emptiness that echoed through my very being.\n\nIt was then that I saw her. A figure in the distance, walking along the path that wound its way through the heath. Her long, dark hair flowed behind her like a river of silk, and her eyes sparkled like diamonds in the sunlight.\n\nMy heart raced as she drew closer, and I found myself transfixed by her beauty. She was like a creature from another world, a being of pure light and grace that had been sent to guide me through the darkness.\n\nAs she approached, I felt a su

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [240]:
import numpy as np
print(len(WANS),np.array(wordcounts).shape)

48 (48, 211)


# convert Chatgpt to dataset

In [241]:
import pickle
import torch_geometric
import torch
import networkx as nx
from networkx import from_numpy_array
from torch_geometric.data import Data,Dataset
from torch_geometric.utils import from_networkx

In [242]:
Graphs = []
for i in range(len(wordcounts)):
    adj_matrix = np.array(WANS[i])
    graph = from_numpy_array(adj_matrix,create_using=nx.DiGraph)
    
    data = from_networkx(graph)
    edge_index = data.edge_index
    edge_attr = data.weight
    signal = torch.tensor(wordcounts[i])
    g = Data(x = signal.reshape(-1,1), edge_index = edge_index,edge_attr = edge_attr,y = torch.tensor([[0]])) # i define chat gpt's label as zero
    Graphs.append(g)
########################################################################
# now include human novels into the dataset
print(len(Graphs))

48


In [243]:
for i in range(len(human_data["all_authors"])):
    print(human_data["all_authors"][i])
    author_excerpts = torch.tensor(human_data["all_wans"][i])
    
    num_papers = author_excerpts.shape[-1]
    for num in range(2):#num_papers):
        adj_matrix = author_excerpts[:,:,num]
        signal = human_data["all_freqs"][i][:,num].reshape(-1,1)
        signal = torch.tensor(signal)
        y = torch.tensor([[1]])
        adj_matrix = np.array(adj_matrix)
        
        graph = from_numpy_array(adj_matrix,create_using=nx.DiGraph)
       
        data = from_networkx(graph)
        edge_index = data.edge_index
        edge_attr = data.weight
        g = Data(x = signal, edge_index = edge_index,edge_attr = edge_attr,y = y) # i define chat gpt's label as zero
        Graphs.append(g)

['abbott']
['stevenson']
['alcott']
['alger']
['allen']
['austen']
['bronte']
['cooper']
['dickens']
['garland']
['hawthorne']
['james']
['melville']
['page']
['thoreau']
['twain']
['doyle']
['irving']
['poe']
['jewett']
['wharton']


In [244]:
# store data into pickle
with open("Data/graph_dataset.txt",'wb') as f:
    pickle.dump(Graphs,f)

# Model GCNN

In [245]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split

In [246]:
train_test = train_test_split(Graphs,train_size = 0.7,test_size = 0.3, shuffle=True)
print(len(Graphs),Graphs[12])

90 Data(x=[211, 1], edge_index=[2, 261], edge_attr=[261], y=[1, 1])


In [249]:
train_loader = DataLoader(train_test[0],batch_size = 16)
test_loader =  DataLoader(train_test[1],batch_size = 16)


In [250]:
for i in train_loader:
    print(i)
    

DataBatch(x=[3376, 1], edge_index=[2, 6385], edge_attr=[6385], y=[16, 1], batch=[3376], ptr=[17])
DataBatch(x=[3376, 1], edge_index=[2, 6230], edge_attr=[6230], y=[16, 1], batch=[3376], ptr=[17])
DataBatch(x=[3376, 1], edge_index=[2, 7060], edge_attr=[7060], y=[16, 1], batch=[3376], ptr=[17])
DataBatch(x=[2954, 1], edge_index=[2, 5443], edge_attr=[5443], y=[14, 1], batch=[2954], ptr=[15])


In [256]:
from torch.nn import Module,Linear,ReLU,Dropout,Sigmoid,ModuleList
from torch_geometric.nn import GCNConv
class GCNN(Module):
    def __init__(self,in_channel,hidden_channels,out_channel,dropout_rate,resize_num =211 ):
        super().__init__()
        self.convs = []
        
        prev_channels = in_channel
        for channel in hidden_channels:
            self.convs.append(GCNConv(prev_channels,channel))
            prev_channels = channel
        self.convs = torch.nn.ModuleList(self.convs)
        self.linear = Linear(prev_channels*resize_num,out_channel)
        self.relu = ReLU()
        self.dropout = Dropout(dropout_rate)
        self.sigmax = Sigmoid()
        
    def forward(self,x,edge_index,edge_attr,batch_size = 128):
        for conv in self.convs:
            
            
            x = conv(x,edge_index,edge_attr)
            x = self.dropout(x)
            x = self.relu(x)
        x = x.reshape(batch_size,-1)
        #print(x.shape)
        x = self.linear(x)
        x = self.sigmax(x)
        return x

            
            
model = GCNN(1,[16,32,64,32,16,1],1,0.3)

    
    
    

In [261]:
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)
def train(model,criteria,optimizer,epoch,train_loader):
    for i in range(epoch):
        losses = 0
        total_count = 0
        accs = 0
        for batch in train_loader:
            #print(batch)
            optimizer.zero_grad()
            x = batch.x
            edge_index = batch.edge_index
            edge_attr = batch.edge_attr
           
            prediction = model.forward(x.float(),edge_index,edge_attr,batch.y.shape[0]).float()
            print(prediction.shape)
            #print(batch.y.shape)
            loss = criteria(prediction,batch.y.float())
            loss.backward()
            optimizer.step()
            #accs = test(model,test_loader)
            losses+=loss
            total_count+=1
            prediction = prediction > 0.5
            
            accs += (prediction == batch.y).sum()/prediction.shape[0]
            
            
        print("epoch ",i," is ",loss/total_count," accs is ", accs/total_count)#," test acc ",accs)

    
train(model,loss,optimizer,200,train_loader)        
        

torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([14, 1])
epoch  0  is  tensor(1.0644e-08, grad_fn=<DivBackward0>)  accs is  tensor(0.9844)
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([14, 1])
epoch  1  is  tensor(9.5822e-05, grad_fn=<DivBackward0>)  accs is  tensor(1.)
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([14, 1])
epoch  2  is  tensor(2.3205e-05, grad_fn=<DivBackward0>)  accs is  tensor(1.)
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([14, 1])
epoch  3  is  tensor(1.7595e-06, grad_fn=<DivBackward0>)  accs is  tensor(1.)
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([14, 1])
epoch  4  is  tensor(3.5725e-05, grad_fn=<DivBackward0>)  accs is  tensor(1.)
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([16, 1])
torch.Size([14, 1])
epoch  5  is  tensor(1.0881e-05, grad_fn=<DivBackward0>)  accs is  tensor(1.)
torch.Size([16, 1])
torch.Size([16, 1])
torch.Si

211.203125